<a href="https://colab.research.google.com/github/Zr695/IMLO-OpenAssessment/blob/main/IMLO_OpenAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from google.colab import runtime

data_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    ])

training_data = datasets.Flowers102(
    root="data",
    split='train',
    download=True,
    transform=data_transform
)
validation_data = datasets.Flowers102(
    root="data",
    split='val',
    download=True,
    transform=ToTensor()
)
test_data = datasets.Flowers102(
    root="data",
    split='test',
    download=True,
    transform=ToTensor()
)


train_dataloader = DataLoader(training_data, batch_size=64)
validate_dataloader = DataLoader(validation_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 125 * 125, 256),
            nn.ReLU(),
            nn.Linear(256, 102)

        )
    def __len__(self):
        return len(self.data)

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork()
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device:{dev}")


Using device:cuda


In [13]:
learning_rate = 0.001
batch_size = 64
epochs = 15

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 2 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
      for X, y in dataloader:
          pred = model(X)
          test_loss += loss_fn(pred, y).item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(validate_dataloader, model, loss_fn)
test_accuracy = test_loop(test_dataloader, model, loss_fn)
print(f"Final Test Accuracy: {test_accuracy:.2f}")
runtime.unassign()

Epoch 1
-------------------------------
loss: 4.632387  [   64/ 1020]
loss: 4.588037  [  192/ 1020]
loss: 4.601827  [  320/ 1020]
loss: 4.607221  [  448/ 1020]
loss: 4.640420  [  576/ 1020]
loss: 4.676603  [  704/ 1020]
loss: 4.626159  [  832/ 1020]
loss: 4.637403  [  960/ 1020]
Test Error: 
 Accuracy: 1.1%, Avg loss: 4.626248 

Epoch 2
-------------------------------
loss: 4.607036  [   64/ 1020]
loss: 4.603019  [  192/ 1020]
